In [1]:
import pandas as pd
import sqlite3
import os
from functools import wraps
from datetime import datetime
import matplotlib.pyplot as plt

# Objetivo do notebook

Este notebook tem por objetivo responder às questões enviadas para o desafio, registrando de forma compreensível por humanos a metodologia e os processamentos que serão implementados nos endpoints da API.

Também registraremos aqui a resposta à última questão, que, conforme instruções enviadas, deve ser respondida separadamente, não entrando no escopo da API.

#### Carregando os dados

Nas células a seguir carregaremos os dados gerados pelo ETL (ver notebook "etl_dados_para_sqlite.ipynb")

In [2]:
path_dados = os.path.abspath(os.path.join('data', 'desafio_selecao.db'))
con = sqlite3.connect(path_dados)

In [3]:
df = pd.read_sql('SELECT * FROM respostas_diagnostico', con)

In [4]:
con.close()

In [5]:
df.head()

,id_resposta,ano_diagnostico,data_submissao,orgao,tipo_orgao,qtd_equipe,utiliza_metodologia,desktop_proprio,desktop_locado,desktop_proprio_antigo


In [6]:
len(df)

0

In [7]:
df.dtypes

id_resposta               object
ano_diagnostico           object
data_submissao            object
orgao                     object
tipo_orgao                object
qtd_equipe                object
utiliza_metodologia       object
desktop_proprio           object
desktop_locado            object
desktop_proprio_antigo    object
dtype: object

Algumas colunas que parecem ser float vieram como text, mas isso está de acordo com o schema e com o dicionário de variáveis.

É possível que tenhamos que tratar isso, no entanto, para responder alguma das questões.

### Filtrando pela data de reposição

Conforme orientações:

> É importante ressaltar também que o campo data_submissao é preenchido apenas quando o formulário é efetivamente concluído e aceito. Por isso, considere somente as respostas que possuam o campo data_submissao preenchido!

In [8]:
sem_data = df['data_submissao'].isnull()

In [9]:
sem_data.mean()

nan

Aproximadamente 30% das respostas nao possui data de submissao

In [10]:
df[sem_data].sample(5)

ValueError: a must be greater than 0 unless no samples are taken

Como podemos ver, os formulário sem data de submissão vieram de fato incompletos.

Vamos inserir portanto esse filtro na função de obtenção dos dados, para garantir que ele seja sempre realizado.

In [ ]:
def get_data():
    
    path_dados = os.path.abspath(os.path.join('data', 'desafio_selecao.db'))
    con = sqlite3.connect(path_dados)
    
    df = pd.read_sql('SELECT * FROM respostas_diagnostico WHERE data_submissao NOT NULL', 
                     con)
    
    return df

In [ ]:
df = get_data()

In [ ]:
assert df['data_submissao'].notnull().all()

Como diversas questões utilizam o parâmetro ano, vamos criar um decorator para limpar esse dado e levantar erros semânticos caso o parâmetro seja imputado incorretamente (por exemplo, caso seja um ano não compreendido na pesquisa).

O mesmo será feito para o parametro secretaria.

Além disso, vamos fazer uma função para sempre copiar o dataframe, evitando que ele seja modificado globalmente.

Como os dados são pequenos, isso nos permitirá fazer uma otimização na API, colocando os dados in memory e apenas copiando o objeto dataframe de thread a thread da API. Isso nos permitirá evitar fazer a leitura dos dados (que é uma ação de I.O. um pouco mais lenta). Note que isso é recomendado apenas para casos como esse, em que a quantidade de dados é pequena. Em outros casos, o ideal é fazer a leitura dos dados a cada chamada de API, e implementar algum tipo de cache.

In [ ]:
def solve_ano(func):
    '''
    Solves the ano param, raising appropriated errors.
    Decorated func must have one keyword arguments named
    ano. Also, df should be the first positional argument'''
    
    @wraps(func)
    def decorated(*args, **kwargs):
        
        if len(args)<1:
            df = kwargs['df']
        else:
            df =  args[0]
        ano = kwargs.pop('ano')
        try:
            ano = int(ano)
            kwargs['ano'] = ano
        except ValueError:
            raise ValueError('Param <ano> must be integer, float or a string convertible to int')

        if ano not in df['ano_diagnostico'].unique():
            raise NotImplementedError(f'Ano {ano} não compreendido na pesquisa')
            
        return func(*args, **kwargs)
    
    return decorated

In [ ]:
def solve_orgao(func):
    '''
    Solves the orgao param, raising appropriated errors.
    Decorated func must have one keyword arguments named
    orgao. Also, df should be the first positional argument'''
    
    @wraps(func)
    def decorated(*args, **kwargs):
        
        if len(args)<1:
            df = kwargs['df']
        else:
            df =  args[0]
        orgao = kwargs.get('orgao')
        if orgao is not None:
            if type(orgao) is not str:
                raise ValueError('Param <orgao> must be string')
            orgao = orgao.upper()

            if orgao not in df['orgao'].unique():
                raise NotImplementedError(f'Orgao {orgao} não compreendido na pesquisa')
            
            kwargs['orgao'] = orgao
            
        return func(*args, **kwargs)
    
    return decorated

In [ ]:
def copy_df(func):
    '''
    Copies the dataframe, avoiding unwanted modifications
    '''
    
    @wraps(func)
    def decorated(*args, **kwargs):
        
        if len(args)<1:
            df = kwargs['df']
            df = df.copy()
            kwargs['df'] = df
        else:
            df =  args[0]
            df = df.copy()
            args = list(args)
            args[0] = df
        
            
        return func(*args, **kwargs)
    
    return decorated

#### Questão 1

> Gostaria de listar os órgãos que responderam a pesquisa, passando o ano referência do diagnóstico como parâmetro. Eles devem ser apresentados como uma lista de objetos e conter minimante o nome do órgão e o tipo de órgão.

In [ ]:
@solve_ano
@copy_df
def questao_1(df, *, ano):
    
    filtro_ano = df['ano_diagnostico'] == ano
    df_ano = df[filtro_ano].reset_index(drop=True)
    
    orgaos_e_tipos = set()
    
    for i, row in df_ano.iterrows():
        
        orgaos_e_tipos.add((row['orgao'], row['tipo_orgao']))
    
    return [{'nome_orgao' : orgao, 'tipo_orgao' : tipo} for orgao, tipo in orgaos_e_tipos]

In [ ]:
questao_1(df, ano = '2019')

In [ ]:
try:
    questao_1(df = df, ano = '2020')
except NotImplementedError as e:
    print(e)

#### Questão 2

>Passando o órgão como parâmetro opcional e ano como parâmetros obrigatório na chamada, gostaria que saber quantas pessoas trabalharam de forma dedicada à TI na Prefeitura de São Paulo.

In [ ]:
@solve_ano
@solve_orgao
@copy_df
def questao_2(df, *,  ano, orgao=None):
    
    if orgao is not None:
    
        df = df[df['orgao']==orgao].reset_index(drop=True)
        
    
    df = df[df['ano_diagnostico'] == ano].reset_index(drop=True)
    
    #certificar que a coluna esta com a tipagm correta
    df['qtd_equipe'] = df['qtd_equipe'].astype(int)
    
    
    return {'qtd_pessoas_dedicadas_a_ti' : df['qtd_equipe'].sum()}

In [ ]:
questao_2(df=df, ano=2019)

In [ ]:
questao_2(df= df, ano = 2019, orgao = 'SMIT')

In [ ]:
try:
    questao_2(df = df, ano = 2005, orgao = 'smit')
except NotImplementedError as e:
    print(e)

In [ ]:
try:
    questao_2(df = df, ano = 2019, orgao = 'SEMPLA')
except NotImplementedError as e:
    print(e)

#### Questão 3

>Considerando que todas as pessoas que trabalharam de forma dedicada a TI receberam R$ 12.500,00/mês, gostaria de saber qual a proporção de custo com pessoal de TI por tipo de órgão.

In [ ]:
@copy_df
def questao_3(df):
    
    SALARIO_TI = 12500
    df['salario'] = SALARIO_TI
    total = df['salario'].sum()
    grouped = df.groupby('tipo_orgao')['salario'].sum()/total
    
    assert grouped.sum()==1
    
    #renomear os labels para ficar no padrao JSON
    grouped = grouped.rename({item : item.lower().replace(' ', '_')
                            for item in grouped.index}, axis=0)
    
    return grouped.to_dict()

In [ ]:
questao_3(df)

#### Questao 4
>Gostaria de listar a quantidade de desktop próprios e desktop locados, por secretaria.

Antes de resolver essa questão, vamos inspecionar o dados.

Pois as colunas estão marcadas como texto no banco e não pudemos convertê-las diretamente para float.

Na célula a seguir, iteramos pelo dataframe para verificar quais valores estão fora do padrão.

In [ ]:
cols = ['desktop_proprio',
           'desktop_locado',
           'desktop_proprio_antigo']

for col in cols:
    for i, row in df.iterrows():
        try:
            float(row[col])
        except:
            print(i, col, f'val: {row[col]}')

In [ ]:
df.loc[48, 'desktop_locado'] == ''

Notamos que o problema está na coluna desktop_locado, que tem alguns valores marcados como strings vazias.

Vamos arrumar isso em nossa função.

In [ ]:
@copy_df
def questao_4_todos_anos(df):
    
    cols = ['desktop_proprio',
           'desktop_locado']    
    #precisa arrumar a tipagem, esta como text no banco
    for col in cols:
        df[col] = df[col].apply(lambda x: int(x) if x != '' else 0)
            
    
    grouped = df.groupby('orgao').sum()[['desktop_locado', 'desktop_proprio']]
    
    
    return grouped

In [ ]:
resp = questao_4_todos_anos(df)

In [ ]:
resp[resp['desktop_locado']==resp['desktop_locado'].max()]

In [ ]:
resp[resp['desktop_proprio']==resp['desktop_proprio'].max()]

No entanto, pensando bem, não faz sentido somar a quantidade de desktops para as respostas de todos os anos. O correto é verificar a quantidade de desktops de uma dada secretaria para um dado ano.

É isso que implementamos abaixo, usando como padrão o ano mais recente da pesquisa.

In [ ]:
@copy_df
def questao_4(df):
    
    
    cols = ['desktop_proprio',
           'desktop_locado']    
    #precisa arrumar a tipagem, esta como text no banco
    for col in cols:
        df[col] = df[col].apply(lambda x: int(x) if x != '' else 0)
    
    ano = df['ano_diagnostico'].max()
    df = df[df['ano_diagnostico']==ano]
    
    grouped = df.groupby('orgao').sum()[['desktop_locado', 'desktop_proprio']]
    
    result = grouped.reset_index()
    
    return result.to_dict(orient='records')

In [ ]:
questao_4(df)

### Questão 5
>Criar uma tabela cópia da tabela respostas_diagnostico apenas com os formulários que foram devidamente concluídos e aceitos. Incluir uma coluna do tipo datetime para gravar a data de última atualização de cada registro.

O primeiro requisito nós já atendemos inicialmente, quando, na função get_data filtramos os dados para obter apenas os formularios submetidos.

Assim, vamos nos valer de nosso dataframe, apenas acrescentando a eles uma coluna de datetime com a data e hora de hoje/agora.

Em seguida, vamos usar a função nativa do pandas para copiar os dados em uma nova tabela de nossa base sqlite3.



In [ ]:
df['dtime_atualizacao'] = datetime.now()

In [ ]:
#certificando que a tipagem está correta
df.dtypes

In [ ]:
con = sqlite3.connect(path_dados)

In [ ]:
df.to_sql('tbl_q_5', con,  if_exists='replace')

In [ ]:
con.commit()

In [ ]:
df_novo = pd.read_sql('SELECT * FROM tbl_q_5', con)

In [ ]:
df_novo.sample(5)

In [ ]:
#ele é incapaz de parsear automaticamente o datetime
#porque o sqlite3 nao suporta esse tipo de dados
df_novo.dtypes

In [ ]:
con.close()

#### Questão 6
>Criar um único endpoint para atualização das seguintes informações na tabela criada no item 5: Quantidade de pessoas que trabalham de forma dedicada, utilização de metodologia para gerenciamento de projetos, desktops próprios, locados e antigos.

Essa questão só pode ser resolvida na implementação da API.

#### Questão 7
>Questão teórica: Considerando que os dados sobre quantidade de pessoas que trabalham dedicadas à Tecnologia da Informação no ano de 2019 apresenta a seguinte distribuição no ano de 2019, complete a frase que aparece abaixo dos histogramas:

>OBS: Os intervalos do histograma são fechados à esquerda (ex: 0 até 1,999...; 2 até 3,999...; etc.)
        
>a) "De acordo com o Diagnóstico de 2019, metade dos órgãos setoriais da prefeitura possui, no máximo, ___ pessoas trabalhando de forma dedicada à área de TI".

In [ ]:
from IPython.display import Image
path_grafico = os.path.join('original_data', 'grafico_q7.png')
Image(filename=path_grafico) 

Primeiro vamos ver se conseguimos reproduzir o gráfico acima com os dados

In [ ]:
df_2019 = df[df['ano_diagnostico']==2019].copy()

In [ ]:
por_orgao = df_2019.groupby('orgao')['qtd_equipe'].sum()

ax = por_orgao.plot.hist(figsize=(10,5), bins=30,
                    title='Distribuição de pessoas dedicadas a TI por órgão em 2019')
plt.xticks(ticks=range(0,61,2))
plt.show()

In [ ]:
texto_questao = 'a) "De acordo com o Diagnóstico de 2019, metade dos órgãos setoriais da prefeitura possui, no máximo, {resp} pessoas trabalhando de forma dedicada à área de TI".'

In [ ]:
mediana = por_orgao.median()

In [ ]:
mediana

In [ ]:
len(por_orgao[por_orgao<4])/len(por_orgao)

In [ ]:
len(por_orgao[por_orgao<=4])/len(por_orgao)

In [ ]:
texto_questao.format(resp=mediana)